# Frequency analysis module - Regional analysis

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import xclim
import xdatasets as xd
from lmoments3.distr import KappaGen
from scipy import stats
from sklearn.cluster import HDBSCAN, OPTICS, AgglomerativeClustering
from sklearn.decomposition import PCA

import xhydro as xh
import xhydro.frequency_analysis as xhfa
import xhydro.gis as xhgis
from xhydro.frequency_analysis.regional import *

This notebook will demonstrate how to use the xhydro package to perform regional frequency analysis on a dataset of streamflow data. The first steps will be similar to the local frequency analysis notebook, but will will keep it simple to focus on the regional frequency analysis.

Lets start with getting the 02 region stations that are natural and have a minimum duration of 15 years

In [ ]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["02*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

ds

Here, we hide years with more than 15% of missing data and get yearly max and spring max


In [ ]:
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "annual": {},
}

ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

ds_4fa

Again the following operations are simillar to the ones performed in the previous notebook.

In [ ]:
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)
ds_4fa

In [ ]:
gdf = xd.Query(
    **{
        "datasets": {
            "deh_polygons": {
                "id": ["02*"],
                "regulated": ["Natural"],
                "variables": ["streamflow"],
            }
        },
        "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
    }
).data.reset_index()
gdf

To do a regional analysis, we'll need some explainatory variables. 
So with those catchments, we can now calculate some of the catchments properties.
We could also get meteorological values and land use data. Refer to GIS example for more details.

In [ ]:
dswp = xhgis.watershed_properties(
    gdf[["Station", "geometry"]], unique_id="Station", output_format="xarray"
)
cent = dswp["centroid"].to_numpy()
lon = [ele[0] for ele in cent]
lat = [ele[1] for ele in cent]
dswp = dswp.assign(lon=("Station", lon))
dswp = dswp.assign(lat=("Station", lat))
dswp = dswp.drop("centroid")
dswp

To do our regional frequency analysis, we'll process the data with a principal component analysis (PCA)

In [ ]:
data, pca = xhfa.regional.fit_pca(dswp, n_components=3)

we can see that the correlation is close to 0 between the components

In [ ]:
data.to_dataframe(name="value").reset_index().pivot(
    index="Station", columns="components"
).corr()

In [ ]:
data

Different clustering methods can be used. Their parameters can be passed as a dict.
We're using AgglomerativeClustering here but later we'll show and exammple using a combination of different methods.

In [ ]:
xhfa.regional.get_group_from_fit(HDBSCAN, {"min_cluster_size": 2}, data)
xhfa.regional.get_group_from_fit(OPTICS, {"min_samples": 2}, data)
groups = xhfa.regional.get_group_from_fit(
    AgglomerativeClustering, {"n_clusters": 3}, data
)
groups

We calculate the L-moments for each station


In [ ]:
ds_moment = xr.apply_ufunc(
    moment_l_vector, ds_4fa, input_core_dims=[["time"]], output_core_dims=[["lmom"]]
).assign_coords(lmom=["l1", "l2", "l3", "tau", "tau3", "tau4"])
ds_moment

We then create groupes of values and moments for each group.

In [ ]:
ds_groups = xr.concat(
    [
        ds_4fa.sel(id=groups[i]).assign_coords(group_id=i).expand_dims("group_id")
        for i in range(len(groups))
    ],
    dim="group_id",
)
ds_moments_groups = xr.concat(
    [
        ds_moment.sel(id=groups[i]).assign_coords(group_id=i).expand_dims("group_id")
        for i in range(len(groups))
    ],
    dim="group_id",
)

For each group, calculate the H and Z values

In [ ]:
kap = KappaGen()
ds_H_Z = calc_h_z(ds_groups, ds_moments_groups, kap)
ds_H_Z

We filter the data to only include the data that has H and Z below the thresholds

In [ ]:
mask = mask_h_z(ds_H_Z)
ds_groups_H1 = ds_groups.where(mask).load()
ds_moments_groups_H1 = ds_moments_groups.where(mask).load()

In [ ]:
# Centiles and return periods :
centiles = [x / 100.0 for x in range(101)]
return_periods = [
    1.010101,
    1.052632,
    1.111111,
    1.25,
    1.5,
    2,
    3,
    5,
    10,
    20,
    50,
    100,
    200,
    500,
    1000,
    2000,
    5000,
    10000,
]

We can now calculate the values for each group and return period and we remove the regions with less stations than a threshold

In [ ]:
Q_T = calculate_rp_from_afr(ds_groups_H1, ds_moments_groups_H1, return_periods)
Q_T = remove_small_regions(Q_T)

To plot, let see what it looks like on 023401

In [ ]:
Q_reg = Q_T.sel(id="023401").dropna(dim="group_id", how="all")
reg = Q_reg.streamflow_max_annual.squeeze()

Let's compare local and regional

In [ ]:
params_loc = xhfa.local.fit(ds_4fa)
Q_loc = xhfa.local.parametric_quantiles(params_loc, return_periods)
loc = Q_loc.sel(id="023401", scipy_dist="genextreme").streamflow_max_annual

In [ ]:
import matplotlib.pyplot as plt

Q_reg
plt.plot(reg.return_period.values, reg.values, "blue")
plt.plot(loc.return_period.values, loc.values, "red")

Now let's add some uncertainities
But we will work with only one catchemnt and two distributions as uncertinities can be intensive in computation.
We selct the station 023401, and distribution 'genextreme' and 'pearson3'

In [ ]:
ds_4fa_one_station = ds_4fa.sel(id="023401")
params_loc_one_station = params_loc.sel(
    id="023401", scipy_dist=["genextreme", "pearson3"]
)

We bootstrap the observations 200 times to get the uncertainty

In [ ]:
ds_4fa_iter = xhfa.uncertainities.boostrap_obs(ds_4fa_one_station, 200)
params_boot_obs = xhfa.local.fit(ds_4fa_iter, distributions=["genextreme", "pearson3"])

In [ ]:
Q_boot_obs = xhfa.local.parametric_quantiles(
    params_boot_obs.load(), return_periods
).squeeze()
Q_boot_obs = Q_boot_obs.streamflow_max_annual

Here, instead of ressampling the observations, we ressamplee the fittted distributions 200 times to get the uncertainty

In [ ]:
values = xhfa.uncertainities.boostrap_dist(
    ds_4fa_one_station, params_loc_one_station, 200
)
params_boot_dist = xhfa.uncertainities.fit_boot_dist(values)

In [ ]:
Q_boot_dist = xhfa.local.parametric_quantiles(
    params_boot_dist.load(), return_periods
).squeeze()
Q_boot_dist = Q_boot_dist.streamflow_max_annual

In [ ]:
loc_dist = Q_boot_dist.sel(scipy_dist="genextreme")
loc_obs = Q_boot_obs.sel(scipy_dist="genextreme")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(reg.return_period.values, reg.values, "blue", label="Regional")
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.5, "samples"),
    "red",
    label="bootstrap obs",
)
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.05, "samples"),
    "pink",
    label="95% CI",
)
plt.plot(loc_obs.return_period.values, loc_obs.quantile(0.95, "samples"), "pink")
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.5, "samples"),
    "green",
    label="bootstrap dist",
)
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.05, "samples"),
    "green",
    label="95% CI",
)
plt.plot(loc_dist.return_period.values, loc_dist.quantile(0.95, "samples"), "green")

For regional, we need to ressample al stations, but this time, it's much faster as no fit is involved

In [ ]:
ds_reg_samples = xhfa.uncertainities.boostrap_obs(ds_4fa, 200)
ds_moments_iter = xhfa.uncertainities.calc_moments_iter(ds_reg_samples).load()

In [ ]:
Q_reg_boot = xhfa.uncertainities.calc_q_iter(
    "023401", "streamflow_max_annual", ds_groups_H1, ds_moments_iter, return_periods
)

In [ ]:
reg_boot = Q_reg_boot.streamflow_max_annual.sel(id="023401")

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 4))
plt.xscale("log")
plt.grid(visible=True)
plt.plot(
    reg_boot.return_period.values,
    reg_boot.quantile(0.5, "samples"),
    "blue",
    label="Regional",
)
plt.plot(
    reg_boot.return_period.values,
    reg_boot.quantile(0.05, "samples"),
    "cyan",
    label="95% CI",
)
plt.plot(reg_boot.return_period.values, reg_boot.quantile(0.95, "samples"), "cyan")
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.5, "samples"),
    "red",
    label="bootstrap obs",
)
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.05, "samples"),
    "pink",
    label="95% CI",
)
plt.plot(loc_obs.return_period.values, loc_obs.quantile(0.95, "samples"), "pink")
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.5, "samples"),
    "green",
    label="bootstrap dist",
)
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.05, "samples"),
    "green",
    label="95% CI",
)
plt.plot(loc_dist.return_period.values, loc_dist.quantile(0.95, "samples"), "green")
plt.legend()

Second we try different clustering methods
We dont do too many tests here since it can take quite a while to run and the goal is just to illustrate the possibilities

In [ ]:
PARAM = {
    AgglomerativeClustering: {"arg_name": "n_clusters", "range": range(2, 12)},
    HDBSCAN: {"arg_name": "min_cluster_size", "range": range(6, 7)},
    OPTICS: {"arg_name": "min_samples", "range": range(4, 5)},
}

So our station instead of beein in one region, will be in many of the regions

In [ ]:
combinations_list = xhfa.uncertainities.generate_combinations(data, 2)

In [ ]:
groups = []

for model in [AgglomerativeClustering, HDBSCAN, OPTICS]:

    for p in PARAM[model]["range"]:
        d_param = {}
        d_param[PARAM[model]["arg_name"]] = p
        for combination in combinations_list:
            # Extract data for the current combination
            data_com = data.sel(Station=list(combination))
            # Get groups from the fit and add to the list
            groups = groups + get_group_from_fit(model, d_param, data_com)
unique_groups = [list(x) for x in {tuple(x) for x in groups}]

The followin steps are similar to the previous one, just with more regions. 

In [ ]:
ds_groups = xr.concat(
    [
        ds_4fa.sel(id=unique_groups[i])
        .assign_coords(group_id=i)
        .expand_dims("group_id")
        for i in range(len(unique_groups))
    ],
    dim="group_id",
)
ds_moments_groups = xr.concat(
    [
        ds_moment.sel(id=unique_groups[i])
        .assign_coords(group_id=i)
        .expand_dims("group_id")
        for i in range(len(unique_groups))
    ],
    dim="group_id",
)

In [ ]:
kap = KappaGen()
ds_H_Z = calc_h_z(ds_groups, ds_moments_groups, kap)

In [ ]:
mask = mask_h_z(ds_H_Z)
ds_groups_H1 = ds_groups.where(mask).load()
ds_moments_groups_H1 = ds_moments_groups.where(mask).load()

Q_T = calculate_rp_from_afr(ds_groups_H1, ds_moments_groups_H1, return_periods)
Q_T = remove_small_regions(Q_T)

Q = Q_T.sel(id="023401").dropna(dim="group_id", how="all")

In [ ]:
regional_multiple_region = Q.streamflow_max_annual

In [ ]:
ds_moment = xr.apply_ufunc(
    moment_l_vector, ds_4fa, input_core_dims=[["time"]], output_core_dims=[["lmom"]]
).assign_coords(lmom=["l1", "l2", "l3", "tau", "tau3", "tau4"])
ds_moment

In [ ]:
fig = plt.figure(figsize=(15, 4))

plt.plot(
    regional_multiple_region.return_period.values,
    regional_multiple_region.quantile(0.5, "group_id"),
    "blue",
    label="regional_multiple_region",
)
plt.plot(
    regional_multiple_region.return_period.values,
    regional_multiple_region.quantile(0.05, "group_id"),
    "cyan",
    label="95% CI",
)
plt.plot(
    regional_multiple_region.return_period.values,
    regional_multiple_region.quantile(0.95, "group_id"),
    "cyan",
)
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.5, "samples"),
    "red",
    label="bootstrap obs",
)
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.05, "samples"),
    "pink",
    label="95% CI",
)
plt.plot(loc_obs.return_period.values, loc_obs.quantile(0.95, "samples"), "pink")
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.5, "samples"),
    "green",
    label="bootstrap dist",
)
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.05, "samples"),
    "green",
    label="95% CI",
)
plt.plot(loc_dist.return_period.values, loc_dist.quantile(0.95, "samples"), "green")
plt.xscale("log")
plt.grid(visible=True)
plt.legend()

We could also combine multiple regions and ressampling.
calc_q_iter will check in how many group_id the station is present, and stack it with samples
In this case, it will be stacked with 200 samples, and it's in 533 groupes so 103600 samples are generated.


In [ ]:
Q_reg_boot = xhfa.uncertainities.calc_q_iter(
    "023401", "streamflow_max_annual", ds_groups_H1, ds_moments_iter, return_periods
)
Q_reg_boot

In [ ]:
regional_multiple_region_boot = Q_reg_boot.sel(id="023401").streamflow_max_annual

In [ ]:
fig = plt.figure(figsize=(15, 4))

plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.5, "samples"),
    "green",
    label="bootstrap dist",
)
plt.plot(
    loc_dist.return_period.values,
    loc_dist.quantile(0.05, "samples"),
    "green",
    label="95% CI",
)
plt.plot(loc_dist.return_period.values, loc_dist.quantile(0.95, "samples"), "green")
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.5, "samples"),
    "red",
    label="bootstrap obs",
)
plt.plot(
    loc_obs.return_period.values,
    loc_obs.quantile(0.05, "samples"),
    "pink",
    label="95% CI",
)
plt.plot(loc_obs.return_period.values, loc_obs.quantile(0.95, "samples"), "pink")
plt.plot(
    regional_multiple_region_boot.return_period.values,
    regional_multiple_region_boot.quantile(0.5, "samples"),
    "black",
    label="regional multiple regions and boot",
)
plt.plot(
    regional_multiple_region_boot.return_period.values,
    regional_multiple_region_boot.quantile(0.05, "samples"),
    "grey",
    label="95% CI",
)
plt.plot(
    regional_multiple_region_boot.return_period.values,
    regional_multiple_region_boot.quantile(0.95, "samples"),
    "grey",
)
plt.plot(
    regional_multiple_region.return_period.values,
    regional_multiple_region.quantile(0.5, "group_id"),
    "blue",
    label="regional multiple regions",
)
plt.plot(
    regional_multiple_region.return_period.values,
    regional_multiple_region.quantile(0.05, "group_id"),
    "cyan",
    label="95% CI",
)
plt.plot(
    regional_multiple_region.return_period.values,
    regional_multiple_region.quantile(0.95, "group_id"),
    "cyan",
)
plt.xscale("log")
plt.grid(visible=True)
plt.legend()